In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.python.framework import ops
import math
from datetime import date
import requests
import json
import urllib

C:\Users\gandh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [75]:
def split_bbox(bbox):
    bbox_org = bbox
    bbox[0] = round(bbox[0] - 0.25, 1)
    bbox[3] = round(bbox[3] + 0.25, 1)
    lat = bbox[2] - bbox[0]
    lon = bbox[3] - bbox[1]
    Loops = max([math.ceil(lat/4.9), math.ceil(lon/4.9)])
#     print(Loops)
    bbox_list = []
    for i in range(math.ceil(lat/4.9)):
        for j in range(math.ceil(lon/4.9)):
            bbox_l = [bbox[0], bbox[1], round(bbox[0]+(lat/math.ceil(lat/4.9)),1), round(bbox[1]+(lon/math.ceil(lon/4.9)),1)]
            bbox_list.append(bbox_l)
            bbox = [bbox[0], round(bbox[1]+(lon/math.ceil(lon/4.9))+0.1,1), bbox[2], bbox[3]]
#         bbox_list.append(bbox)
        bbox[0] = round(bbox[0]+(lat/math.ceil(lat/4.9)),1)
        bbox[2] = round(bbox[0]+(lat/math.ceil(lat/4.9)),1)
        bbox[1] = bbox_org[1]
        bbox[3] = bbox_org[3]
    
    return bbox_list

def create_features(features_list):
    features = ''
    for feature in range(len(features_list)):
        if feature == 0:
            features += str(features_list[feature])
        else:
            features += ',' + str(features_list[feature]) 
    return features   

def create_Dates(start, end): #MM-DD-YYYY
    start = str(start).split('-')
    start_date = ''
    for k in range(-1,len(start)-1):
        start_date += start[k]
    end = str(end).split('-')
    end_date = ''
    for l in range(-1,len(end)-1):
        end_date += end[l]
    dates = [start_date, end_date]
    date = '&startDate=' + dates[0] + '&endDate=' + dates[1]
    return date

In [76]:
Texas_bbox = [25.8, -106.6, 36.5, -93.5]
Texas_bbox_list = split_bbox(Texas_bbox)

In [77]:
Texas_bbox_list

[[25.6, -106.6, 29.2, -102.1],
 [25.6, -102.0, 29.2, -97.5],
 [25.6, -97.4, 29.2, -92.9],
 [29.2, -106.6, 32.8, -102.1],
 [29.2, -102.0, 32.8, -97.5],
 [29.2, -97.4, 32.8, -92.9],
 [32.8, -106.6, 36.4, -102.1],
 [32.8, -102.0, 36.4, -97.5],
 [32.8, -97.4, 36.4, -92.9]]

In [78]:
def Download_Data(bbox_list, features, dates, user_community, temp_Average, Folder_name):
    for j in range(len(bbox_list)):
        url = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=Regional&parameters='\
        + features + dates + \
        '&userCommunity=' + user_community + '&tempAverage=' + temp_Average + '&outputList=CSV&bbox=' +\
        str(bbox_list[j][0]) + ',' + str(bbox_list[j][1]) + ',' + str(bbox_list[j][2]) + ',' + \
        str(bbox_list[j][3]) + '&user=anonymous'
        response = requests.get(url)
        csv_url = response.json()['outputs']['csv']
        urllib.request.urlretrieve(csv_url, 'Data/' + Folder_name + '/File_' + str(j+1) + '.csv')
        print('set ' + str(j+1) + ' is complete' )
    

In [79]:
solar_features = create_features(['PRECTOT', 'RH2M', 'PS', 'T2M', 'ALLSKY_SFC_SW_DWN'])
solar_dates = create_Dates('01-01-2015', '12-31-2017')
Download_Data(Texas_bbox_list, solar_features, solar_dates, 'SB', 'DAILY', 'Solar')

set 1 is complete
set 2 is complete
set 3 is complete
set 4 is complete
set 5 is complete
set 6 is complete
set 7 is complete
set 8 is complete
set 9 is complete


In [80]:
def read_csv_file(filepath, sep=', ', delimiter=',', header='infer'):
    return pd.read_csv(filepath, sep=sep, delimiter=delimiter, header=header)

def Check(df):
    print('shape:' ,df.shape)
    return df.head()

def Get_csv(bbox_list, Folder_name, header):
    data = []
    for k in range(len(bbox_list)):
        data.append(read_csv_file('Data/' + Folder_name + '/File_' + str(k+1) + '.csv', header=header))
    df = pd.concat(data).reset_index(drop=True)
    try:
        df = df.sort_values(['DY', 'MO', 'YEAR','LAT', 'LON'], axis=0).reset_index(drop=True)
    except:
        df = df.sort_values(['LAT', 'LON'], axis=0).reset_index(drop=True)
    df = df.drop_duplicates().reset_index(drop=True)
    return df

def Add_Date_Column(df):
    Date = pd.DataFrame({'year': df['YEAR'],
                       'month': df['MO'],
                       'day': df['DY']})
    Date = pd.to_datetime(Date)
    df['DATE'] = Date
    return df

def Replace_witn_nan(df, null_denoter):
    for i in null_denoter:
        df = df.replace(to_replace=i, value=np.nan)
    return df

def Check_NAN(df):
    return df.isnull().sum()


In [81]:
solar_data = Get_csv(Texas_bbox_list, 'Solar', 14)

solar_data['LON'] = solar_data.LON.apply(lambda x:-x)

solar_data = Add_Date_Column(solar_data)

solar_data = Replace_witn_nan(solar_data, [-999.0, 'nan', "NAN", 'NA'])

Check_NAN(solar_data)

LAT                     0
LON                     0
YEAR                    0
MO                      0
DY                      0
PRECTOT               638
RH2M                    0
PS                      0
T2M                     0
ALLSKY_SFC_SW_DWN    3251
DATE                    0
dtype: int64

In [82]:
def Fill_Null_vals(df):
    try:
        df = df.fillna(method='ffill')
    except:
        df = df.fillna(method='bfill')
    return(df)



In [83]:
solar_data = Fill_Null_vals(solar_data)

Check_NAN(solar_data)

LAT                  0
LON                  0
YEAR                 0
MO                   0
DY                   0
PRECTOT              0
RH2M                 0
PS                   0
T2M                  0
ALLSKY_SFC_SW_DWN    0
DATE                 0
dtype: int64

In [84]:
degree_celcius =  u"\u2103"
degree = u'\xb0'
square = '\u00b2'

In [85]:
def Drop_Column(df,column):
    df = df.drop(column, axis=1)
    return df

solar_data.columns = ['Latitude (' + degree + 'N)', 'Longitude (' + degree + 'W)', 'Year', 'Month', 'Day', 'Precepitation (mm/day)', \
                    'Relative Humidity at 2m (%)', 'Surface Pressure (kPa)', \
                    'Temperature at 2m (' + str(degree_celcius) + ')', \
                    'All Sky Insolation (kW-hr/m' + square + '/day)', 'Date']


In [86]:

solar_data = Drop_Column(solar_data, ['Year', 'Month', 'Day'])

Check(solar_data)

shape: (699248, 8)


,Latitude (°N),Longitude (°W),Precepitation (mm/day),Relative Humidity at 2m (%),Surface Pressure (kPa),Temperature at 2m (℃),All Sky Insolation (kW-hr/m²/day),Date
0,25.75,106.75,1.07,85.70,80.09,8.38,2.18,2015-01-01
1,25.75,106.25,1.34,86.07,76.39,7.25,2.18,2015-01-01
2,25.75,105.75,0.55,77.26,78.32,9.07,2.82,2015-01-01
3,25.75,105.25,0.15,70.79,80.56,10.56,2.82,2015-01-01
4,25.75,104.75,0.01,66.00,82.43,11.58,1.79,2015-01-01


In [87]:
solar_data = solar_data.drop_duplicates()

Check(solar_data)

shape: (699248, 8)


,Latitude (°N),Longitude (°W),Precepitation (mm/day),Relative Humidity at 2m (%),Surface Pressure (kPa),Temperature at 2m (℃),All Sky Insolation (kW-hr/m²/day),Date
0,25.75,106.75,1.07,85.70,80.09,8.38,2.18,2015-01-01
1,25.75,106.25,1.34,86.07,76.39,7.25,2.18,2015-01-01
2,25.75,105.75,0.55,77.26,78.32,9.07,2.82,2015-01-01
3,25.75,105.25,0.15,70.79,80.56,10.56,2.82,2015-01-01
4,25.75,104.75,0.01,66.00,82.43,11.58,1.79,2015-01-01


In [88]:
def Save_as_CSV(df, Folder_name, File_name):
    df.to_csv('Data/' + Folder_name + '/' + File_name + '.csv', index=False)



In [89]:
Save_as_CSV(solar_data, 'Solar', '2015 to 2017')

In [90]:
solar_data = pd.read_csv('Data/Solar/2015 to 2017.csv')
solar_data#.groupby(solar_data.index // 365 * 365).sum().reset_index(drop=True)
# max(solar_data['All Sky Insolation (kW-hr/m' + square + '/day)'])

,Latitude (°N),Longitude (°W),Precepitation (mm/day),Relative Humidity at 2m (%),Surface Pressure (kPa),Temperature at 2m (℃),All Sky Insolation (kW-hr/m²/day),Date
0,25.75,106.75,1.07,85.70,80.09,8.38,2.18,2015-01-01
1,25.75,106.25,1.34,86.07,76.39,7.25,2.18,2015-01-01
2,25.75,105.75,0.55,77.26,78.32,9.07,2.82,2015-01-01
3,25.75,105.25,0.15,70.79,80.56,10.56,2.82,2015-01-01
4,25.75,104.75,0.01,66.00,82.43,11.58,1.79,2015-01-01
5,25.75,104.25,0.00,63.17,84.14,12.12,1.79,2015-01-01
6,25.75,103.75,0.00,63.41,86.69,12.34,2.30,2015-01-01
7,25.75,103.25,0.01,66.93,87.90,12.07,2.30,2015-01-01
8,25.75,102.75,0.05,67.10,87.87,13.05,1.95,2015-01-01
9,25.75,102.25,0.06,66.55,86.92,13.85,1.95,2015-01-01


In [91]:
# solar_data = pd.read_csv('Global Radiation for Pittsburgh.csv')

def convert_to_date(x):
    return datetime.strptime(x, '%Y-%m-%d')

solar_data.Date = solar_data.Date.apply(lambda x: convert_to_date(x)) 

In [92]:
year_2015 = solar_data[solar_data['Date'].dt.year==2015].iloc[:,2:-1].reset_index(drop=True)
year_2016 = solar_data[solar_data['Date'].dt.year==2016].iloc[:,2:].reset_index(drop=True)
year_2017 = solar_data[solar_data['Date'].dt.year==2017].iloc[:,2:-1].reset_index(drop=True)

In [93]:
feb_2016 = year_2016[year_2016['Date'].dt.month==2]
feb_29_2016 = feb_2016[feb_2016['Date'].dt.day==29]
drop_cols = list(feb_29_2016.index)
# year_2016['Date'][60000].month, year_2016['Date'][60000].day#==datetime.date(2016, 7, 22)

In [94]:
if len(year_2016) != len(year_2015):
    year_2016 = year_2016.drop(drop_cols,axis=0)
    year_2016 = year_2016.drop('Date',axis=1)

In [95]:
len(year_2015), len(year_2016), len(year_2017)

(232870, 232870, 232870)

In [96]:
average = (year_2015.as_matrix()+year_2016.as_matrix()+year_2017.as_matrix())/3

C:\Users\gandh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [97]:
solar_average = pd.DataFrame(data = average, columns=year_2015.columns)
# solar_average['day of the year'] = solar_average.index+1

solar_average['Lat'] = solar_data[solar_data['Date'].dt.year==2015].reset_index(drop=True).iloc[:,0]
solar_average['Lon'] = solar_data[solar_data['Date'].dt.year==2015].reset_index(drop=True).iloc[:,1]
solar_average['Day'] = solar_data[solar_data['Date'].dt.year==2015].reset_index(drop=True).iloc[:,-1].dt.dayofyear
solar_average

,Precepitation (mm/day),Relative Humidity at 2m (%),Surface Pressure (kPa),Temperature at 2m (℃),All Sky Insolation (kW-hr/m²/day),Lat,Lon,Day
0,0.683333,83.976667,80.190000,8.256667,2.570000,25.75,106.75,1
1,1.350000,85.146667,76.476667,6.336667,2.570000,25.75,106.25,1
2,1.673333,76.770000,78.420000,8.256667,2.810000,25.75,105.75,1
3,1.993333,70.640000,80.673333,9.986667,2.810000,25.75,105.25,1
4,2.756667,66.670000,82.566667,11.156667,2.226667,25.75,104.75,1
5,4.026667,64.576667,84.286667,11.843333,2.226667,25.75,104.25,1
6,5.463333,64.056667,86.853333,12.593333,2.386667,25.75,103.75,1
7,5.896667,63.480000,88.060000,12.896667,2.386667,25.75,103.25,1
8,5.023333,63.860000,88.040000,13.116667,2.520000,25.75,102.75,1
9,4.846667,64.743333,87.090000,12.913333,2.520000,25.75,102.25,1


In [98]:
solar_average.to_csv('Data/Solar/Average.csv', index=False)

In [99]:
Surface_Albedo = 'SRF_ALB'
Equivalent_Number_Of_NO_SUN = 'EQVLNT_NO_SUN_BLACKDAYS_1'